<a href="https://colab.research.google.com/github/NadAAaHH/Arabic-Error-Checker/blob/main/Allam_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.0/140.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-ibm to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install --upgrade pip setuptools wheel

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
  Using cached setuptools-75.1.0-py3-none-any.whl.metadata (6.9 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
Using cached setuptools-75.1.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 71.0.4
    Uninstalling setuptools-71.0.4:
      Successfully uninstalled setuptools-71.0.4
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [1]:
import os
import pandas as pd
import getpass
from sklearn.model_selection import train_test_split

# Evaluation metrices
from sklearn.metrics import accuracy_score  # Or any other metric like BLEU, ROUGE
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score


# langchain
import matplotlib.pyplot as plt
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain_ibm import WatsonxEmbeddings
from langchain_ibm import WatsonxLLM
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain


# ibm_watsonx_ai
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

# 1. Loading CSVs and chunking with LangChain



In [68]:
# Simple method - Split by pages
loader = CSVLoader("/content/الأخطاء اللغوية الشائعة (تنظيف).csv")
pages = loader.load_and_split()

print(pages[1])

# SKIP TO STEP 2 IF YOU'RE USING THIS METHOD
chunks = pages

page_content='﻿Error: أَجِبْ عَلى الأسئلة الآتية.
Correction: أَجِبْ عَن الأسئلة الآتية.
Explanation: " أجب عن " بدلاً من " أجب على " لأن الإجابة عن الأسئلة تعني تحديد إجابتها المطلوبة، ولكن "على الأسئلة تعني ترك الأسئلة والإجابة عن غيرها، خاصةً أن حرفَ الجر "على" يعني المُجاوزة وترك الشيء كما هو معروف. وهذا الخطأ يتكرر كثيراً في الاختبارات، لذا يجب أن يتنبه الَّذِين يَضَعُونَ الاختبارات لهذا الأمر. قال تعالى : ومَنْ يَبْخَلْ فإنّما يبخلُ عَنْ نَفْسِهِ" (?) فاستخدم الحق سبحانه يبخلُ" عن" ولم يستخدم يبخل على والفرق واضح بين الاثنين بناءً على ما تقدم.' metadata={'source': '/content/الأخطاء اللغوية الشائعة (تنظيف).csv', 'row': 1}


# 2. Embed text and store embeddings

### Seting up connection with watsonx.ai and get environment Variables

Get watsonx URL

In [3]:
try:
    os.environ["WATSONX_URL"] = wxa_url
except NameError:
    wxa_url = getpass.getpass("Please enter your watsonx.ai URL domain (hit enter): ")
    os.environ["WATSONX_URL"] = wxa_url

Please enter your watsonx.ai URL domain (hit enter): ··········


Get watsonx API key

In [4]:
try:
     os.environ["WATSONX_APIKEY"] = wxa_api_key
except NameError:
    wxa_api_key = getpass.getpass("Please enter your watsonx.ai API key (hit enter): ")
    os.environ["WATSONX_APIKEY"] = wxa_api_key

Please enter your watsonx.ai API key (hit enter): ··········


Get watsonx Project ID

In [5]:
try:
    os.environ["WATSONX_PROJECT_ID"] = wxa_project_id
except NameError:
    wxa_project_id = getpass.getpass("Please enter your watsonx.ai Project ID (hit enter): ")
    os.environ["WATSONX_PROJECT_ID"] = wxa_project_id

Please enter your watsonx.ai Project ID (hit enter): ··········


### Loading the embedding model

In [69]:
# adjust model parameters
embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

In [70]:
multilingual_e5_embedding = WatsonxEmbeddings(
    model_id="intfloat/multilingual-e5-large",
    params=embed_params,
    url=wxa_url,
    project_id=wxa_project_id
)

### Initilaize FIASS vector database

In [71]:
# Create vector database
db = FAISS.from_documents(chunks, multilingual_e5_embedding)

# 3. Setup retrieval function

## Loading ALLAM model

In [72]:
# adjust model parameters
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 500
}

In [73]:
allam_llm = WatsonxLLM(
    model_id="sdaia/allam-1-13b-instruct",
    params=parameters,
    url=wxa_url,
    project_id=wxa_project_id
)

### Get response from ALLAM model directly

In [123]:
Sentence = "معلموا المبحث نشيطون."

In [132]:
Sentence = input("Please enter a sentence with error: ")

Please enter a sentence with error: أعددت وسائلاً كثيرةً.


In [133]:
# define input query
query = "قم بتصحيح الجملة التالية مع توضيح الخطأ اللغوي فيها واكتب الرد باللغة العربية: " + Sentence

In [134]:
query = query = "قم بتصحيح الجملة التالية (" + Sentence + ") مع توضيح الخطأ اللغوي فيها واكتب الرد باللغة العربية. اكتب الرد على النحو التالي:\nالخطأ: الجملة\nالصواب: تصحيح الجملة\nالتفسير: كتابة التفسير"

In [135]:
allam_response = allam_llm(query)

In [136]:
print("Query: " + query)
print("Response: " + allam_response)

Query: قم بتصحيح الجملة التالية (أعددت وسائلاً كثيرةً.) مع توضيح الخطأ اللغوي فيها واكتب الرد باللغة العربية. اكتب الرد على النحو التالي:
الخطأ: الجملة
الصواب: تصحيح الجملة
التفسير: كتابة التفسير
Response:  هنا

الخطأ: الجملة "أعددت وسائلاً كثيرةً."
الصواب: تصحيح الجملة "أعددت وسائلَ كثيرةً."
التفسير: في هذه الجملة، يجب استخدام "وسائل" بدلاً من "وسائلاً" لأن "وسائل" هي جمع تكسير للكلمة "وسيلة" وهي تعبر عن مجموعة من الأشياء أو الأدوات. بالإضافة إلى ذلك، يجب وضع الهمزة على الياء في "كثيرةً" لتصبح "كثيرةً" بدلاً من "كثيرةً" لأن الهمزة يجب أن تكون على الياء في هذه الحالة. 


### Get response from knowledge base (Create QA chain to integrate similarity search with input queries)

In [137]:
# Check that similarity search is working
docs = db.similarity_search(query, k=10)
for i in docs:
  print(i)
  print("-------------")

page_content='﻿Error: أعددت وسائلاً كثيرةً.
Correction: أعددْتُ وسائل كثيرة.
Explanation: لا يصح تنوين الممنوع من الصرف كما مرَّ "وسائل" وهذه الكلمة صيغة منتهى الجموع، وهذه الصيغة ممنوعة من الصرف أي التنوين فترفع بالضمة في حالة الرفع، وتنصب بالفتحة في حالة النصب وتجر بالفتحة في حالة الجر إلا إذا أضيفت أو عُرّفت بأل عندئذٍ تجر بالكسرة وهي الإعراب العادي الطبيعي. 
تقول : أُعجبتُ بالوسائل الحديثة. 
وتقول : أُعجبْتُ بوسائل المصادر التعليمية.' metadata={'source': '/content/الأخطاء اللغوية الشائعة (تنظيف).csv', 'row': 4}
-------------
page_content='﻿Error: قام المعلمين بشرح الفكرة المناسبة.
Correction: قام المعلمون بشرح الفكرة المناسبة.
Explanation: إذا جاءَ جمع المذكر السالم مرفوعاً يرفع بالواو، وإذا جاء منصوباً ينصب بالياء، وكذلك الجر، ولكن غير العارف بقواعد اللغة العربية ربما لا يدرك هذه الحقيقة فيجعل ما هو منصوب مرفوعاً أو العكس. قال تعالى: "إنما المؤمنون إخوة"، وجاء في تعريف جمع المذكر السالم في الألفية:
وارْفَعْ بِواو وَبَيا أجرر وأنْصب * سَالَم جمع عَامِرٍ ومُذْنبِ والفكرة : مفرد وجمع

In [138]:
chain = load_qa_chain(allam_llm, chain_type="stuff")
allam_response_RAG = chain.run(input_documents=docs, question=query)

In [139]:
print("Query: " + query)
print("Response: " + allam_response_RAG)

Query: قم بتصحيح الجملة التالية (أعددت وسائلاً كثيرةً.) مع توضيح الخطأ اللغوي فيها واكتب الرد باللغة العربية. اكتب الرد على النحو التالي:
الخطأ: الجملة
الصواب: تصحيح الجملة
التفسير: كتابة التفسير
Response: 

الخطأ: أعددت وسائلاً كثيرةً.
الصواب: أعددْتُ وسائل كثيرة.
التفسير: لا يصح تنوين الممنوع من الصرف كما مرَّ "وسائل" وهذه الكلمة صيغة منتهى الجموع، وهذه الصيغة ممنوعة من الصرف أي التنوين فترفع بالضمة في حالة الرفع، وتنصب بالفتحة في حالة النصب وتجر بالفتحة في حالة الجر إلا إذا أضيفت أو عُرّفت بأل عندئذٍ تجر بالكسرة وهي الإعراب العادي الطبيعي. تقول : أُعجبتُ بالوسائل الحديثة. وتقول : أُعجبْتُ بوسائل المصادر التعليمية. 


### Paraphrase the Original Query

In [88]:
# define input query
query = "أعد صياغة الجملة التالية بخمس طرق غير متشابهة واجعل كل جملة في سطر جديد  : " + Sentence

In [89]:
allam_response_paraphrase = allam_llm(query)

In [90]:
print("Query: " + query)
print("Response: " + allam_response_paraphrase)

Query: أعد صياغة الجملة التالية بخمس طرق غير متشابهة واجعل كل جملة في سطر جديد  : أَجِبْ عَلى الأسئلة الآتية.
Response:  


# 4. Evaluation

In [ ]:
# Function to find match in original file and return corrrections
def get_ground_truth(csv_path, search_value):
  df = pd.read_csv(csv_path)
  # Filter the DataFrame based on column A value
  match_val = df[df['Error'] == search_value]
  df['Combined'] = match_val['Correction']+ " " + match_val['Explanation']
  return df['Combined'].dropna().tolist()[0]

'\n  # Concatenate columns Correction and Explanation\n  if not match_val.empty:\n    #result = match_val.apply(lambda row: f"{row[\'Correction\']} {row[\'Explanation\']}", axis=1)\n    result = match_val[\'Correction\']+ " " + match_val[\'Explanation\']\n    return result\n  else:\n    return "No match found."\n  '

In [ ]:
# Get ground truth
real_response = get_ground_truth("/content/الأخطاء اللغوية الشائعة (تنظيف).csv", query.split(":")[1].strip())

In [ ]:
real_response

'الطالبات كتبْنَ الدرس. الطالبات مبتدأ مرفوع وهو جمع مؤنث سالم ثم الفعل الذي يليه يجب أن يُسند لنون النسوة وليس إلى واو الجماعة كما ورد في الخطأ السابق. والفعل كتبن: فعل ماض مبني على السكون لاتصاله بنون النسوة، والنون ضمير مبني في محل رفع الفاعل والجملة من الفعل والفاعل في محل رفع خبر المبتدأ" الطالبات وجاء في تعريف المبتدأ: \r\nمُبتدأً زَيْدٌ وَعادَرٌ خَبَرْ * إنْ قُلْتَ زِيدٌ عَاذِرٌ مَنِ اعْتذَرْ \r\nوأول مبتدأ والثاني* فاعل أغنى في : أسارٍ ذانِ؟'

In [ ]:
allam_response_RAG_modified = allam_response_RAG.split("Explanation: ", 1)[1].strip()
allam_response_RAG_modified

'في الجملة الأصلية، تم استخدام "كتبوا" بدلاً من "كتبْنَ" مع الطالبات. الفعل "كتبوا" هو فعل ماضٍ للمذكر الغائب، ولكن في هذه الجملة، الفاعل هم الطالبات، وهن إناث. لذا يجب استخدام "كتبْنَ" بدلاً من "كتبوا" لأن "كتبْنَ" هو فعل ماضٍ للمذكر الغائب المؤنث. الطالبات كتبْنَ الدرس.'

Define Accuracy (Exact Match)

In [ ]:
def calculate_accuracy(predictions, targets):
    correct = sum([1 if pred == target else 0 for pred, target in zip(predictions, targets)])
    accuracy = correct / len(targets)
    return accuracy

In [ ]:
# Calculate accuracy
accuracy_score = calculate_accuracy(allam_response_RAG_modified, real_response)
print("Accuracy Score:", accuracy_score)

Accuracy Score: 0.03571428571428571


BLEU Score (Bilingual Evaluation Understudy)

In [ ]:
def calculate_bleu(predictions, targets):
    scores = [sentence_bleu([target.split()], pred.split()) for pred, target in zip(predictions, targets)]
    return sum(scores) / len(scores)

In [ ]:
# Calculate BLEU score
bleu_score = calculate_bleu(allam_response_RAG_modified, real_response)
print("BLEU Score:", bleu_score)

BLEU Score: 6.050364540593388e-233


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

ROUGE Score

In [ ]:
def calculate_rouge(predictions, targets):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = [scorer.score(target, pred) for pred, target in zip(predictions, targets)]
    avg_rouge = {
        'rouge1': sum([score['rouge1'].fmeasure for score in scores]) / len(scores),
        'rougeL': sum([score['rougeL'].fmeasure for score in scores]) / len(scores),
    }
    return avg_rouge

In [ ]:
# Calculate ROUGE scores
rouge_results = calculate_rouge(allam_response_RAG_modified, real_response)
print(f"Average ROUGE-1 F1 Score: {rouge_results['rouge1']}")
print(f"Average ROUGE-L F1 Score: {rouge_results['rougeL']}")

Average ROUGE-1 F1 Score: 0.0
Average ROUGE-L F1 Score: 0.0


# 5. Hyperparameter Tuning

Define test dataset

In [ ]:
# Load the CSV file
df = pd.read_csv('/content/الأخطاء اللغوية الشائعة (تنظيف).csv')

# Display the first few rows to ensure it's loaded correctly
df.head()

,Error,Correction,Explanation
0,أُكتب المُلخص السبوري.,أكتب المُلخص السبوري.,"الفعل "" اكتب "" فعل أمر للفعل الثلاثي للمخاطب ا..."
1,أَجِبْ عَلى الأسئلة الآتية.,أَجِبْ عَن الأسئلة الآتية.,""" أجب عن "" بدلاً من "" أجب على "" لأن الإجابة عن..."
2,ملائمة الأهداف للخطة الموضوعة.,ملاءمة الأهداف للخطة الموضوعة.,"من الواضح أن كلمة "" ملاءمة"" بها همزة قبلها ألف..."
3,معلموا المبحث نشيطون.,معلمو المبحث نشيطون.,"كلمة ""معلمو"" أصلها ""معلمون"" اسم فاعل للفعل ""عَ..."
4,أعددت وسائلاً كثيرةً.,أعددْتُ وسائل كثيرة.,"لا يصح تنوين الممنوع من الصرف كما مرَّ ""وسائل""..."


In [ ]:
# Merge the correction and explanation columns into a new column
df['Correction_with_Explanation'] = df['Correction'] + " - " + df['Explanation']
df.head()

,Error,Correction,Explanation,Correction_with_Explanation
0,أُكتب المُلخص السبوري.,أكتب المُلخص السبوري.,"الفعل "" اكتب "" فعل أمر للفعل الثلاثي للمخاطب ا...","أكتب المُلخص السبوري. - الفعل "" اكتب "" فعل أمر..."
1,أَجِبْ عَلى الأسئلة الآتية.,أَجِبْ عَن الأسئلة الآتية.,""" أجب عن "" بدلاً من "" أجب على "" لأن الإجابة عن...","أَجِبْ عَن الأسئلة الآتية. - "" أجب عن "" بدلاً ..."
2,ملائمة الأهداف للخطة الموضوعة.,ملاءمة الأهداف للخطة الموضوعة.,"من الواضح أن كلمة "" ملاءمة"" بها همزة قبلها ألف...",ملاءمة الأهداف للخطة الموضوعة. - من الواضح أن ...
3,معلموا المبحث نشيطون.,معلمو المبحث نشيطون.,"كلمة ""معلمو"" أصلها ""معلمون"" اسم فاعل للفعل ""عَ...","معلمو المبحث نشيطون. - كلمة ""معلمو"" أصلها ""معل..."
4,أعددت وسائلاً كثيرةً.,أعددْتُ وسائل كثيرة.,"لا يصح تنوين الممنوع من الصرف كما مرَّ ""وسائل""...",أعددْتُ وسائل كثيرة. - لا يصح تنوين الممنوع من...


In [ ]:
# Split the dataset (80% training, 20% testing)
train_df, test_df = train_test_split(df, test_size=0.6, random_state=42)

# Display the size of the splits
print("Training set size:", len(train_df))
print("Testing set size:", len(test_df))

Training set size: 33
Testing set size: 51


In [ ]:
# Save the testing set to a new CSV file
test_df.to_csv('test_set.csv', index=False)

In [ ]:
test_queries = df['Explanation'].tolist()
correct_responses = df['Correction'].tolist()

In [ ]:
# Generate model predictions for each error
predictions = []
for error in test_queries:
    response = allam_llm(error)
    predictions.append(response)

# Calculate ROUGE scores
rouge_scores = evaluate_rouge(predictions, expected_corrections)

# Print ROUGE scores for each prediction
for i, score in enumerate(rouge_scores):
    print(f"Error: {test_errors[i]}")
    print(f"Expected Correction: {expected_corrections[i]}")
    print(f"Model Correction: {predictions[i]}")
    print(f"ROUGE Score: {score}")
    print("-------------")

# Optionally, calculate average ROUGE score
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

print(f"Average ROUGE-1 F1 Score: {avg_rouge1}")
print(f"Average ROUGE-L F1 Score: {avg_rougeL}")

In [ ]:
# Define hyperparameter space to tune
hyperparameter_grid = {
    'DECODING_METHOD': [DecodingMethods.GREEDY, DecodingMethods.SAMPLE],  # Decoding methods
    'MAX_NEW_TOKENS': [100, 300, 500],  # Maximum number of tokens in response
    'MIN_NEW_TOKENS': [1, 5],  # Minimum number of tokens in response
    'TRUNCATE_INPUT_TOKENS': [3, 4],  # Truncate input tokens for embeddings
}

In [ ]:
# Store results for different hyperparameters
results = []

In [ ]:
# Loop over hyperparameter combinations
for decoding_method in hyperparameter_grid['DECODING_METHOD']:
    for max_new_tokens in hyperparameter_grid['MAX_NEW_TOKENS']:
        for min_new_tokens in hyperparameter_grid['MIN_NEW_TOKENS']:
            for truncate_tokens in hyperparameter_grid['TRUNCATE_INPUT_TOKENS']:

                        # Set up LLM with current hyperparameters
                        parameters = {
                            GenParams.DECODING_METHOD: decoding_method,
                            GenParams.MAX_NEW_TOKENS: max_new_tokens,
                            GenParams.MIN_NEW_TOKENS: min_new_tokens
                        }

                        # Initialize Watson LLM with hyperparameters
                        allam_llm = WatsonxLLM(
                            model_id="sdaia/allam-1-13b-instruct",
                            params=parameters,
                            url=wxa_url,
                            project_id=wxa_project_id
                        )

                        # Set up embeddings with truncate tokens and return options
                        embed_params = {
                            EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: truncate_tokens,
                        }

                        multilingual_e5_embedding = WatsonxEmbeddings(
                            model_id="intfloat/multilingual-e5-large",
                            params=embed_params,
                            url=wxa_url,
                            project_id=wxa_project_id
                        )

                        # Rebuild FAISS vector database with the new embeddings
                        db = FAISS.from_documents(chunks, multilingual_e5_embedding)

                        # Evaluate the model
                        predictions = []
                        for query in test_queries:
                            # Retrieve top-k documents
                            docs = db.similarity_search(query, k=10)

                            # Run QA chain or just the model
                            response = chain.run(input_documents=docs, question=query)
                            predictions.append(response)

                        # Calculate accuracy (or any other metric like BLEU)
                        accuracy = accuracy_score(correct_responses, predictions)

                        # Store the results for this combination
                        results.append({
                            'DECODING_METHOD': decoding_method,
                            'MAX_NEW_TOKENS': max_new_tokens,
                            'MIN_NEW_TOKENS': min_new_tokens,
                            'TRUNCATE_INPUT_TOKENS': truncate_tokens,
                            'accuracy': accuracy
                        })

                        print(f"Tested: {decoding_method}, MAX_NEW_TOKENS: {max_new_tokens}, "
                              f"MIN_NEW_TOKENS: {min_new_tokens}, TRUNCATE_INPUT_TOKENS: {truncate_tokens}, "
                              f"Accuracy: {accuracy}")

Status code: 400, body: {"errors":[{"code":"invalid_request_entity","message":"Failed to deserialize json: 'invalid character 'N' looking for beginning of value'.","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"36af5b549a53deb3609ee3c5dd928294","status_code":400}


ApiRequestFailure: Failure during generate. (POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/embeddings?version=2024-10-01)
Status code: 400, body: {"errors":[{"code":"invalid_request_entity","message":"Failed to deserialize json: 'invalid character 'N' looking for beginning of value'.","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"36af5b549a53deb3609ee3c5dd928294","status_code":400}

In [ ]:
# Initialize the Watsonx foundation model
allam_llm_1 = Model(
    model_id='ibm/granite-20b-multilingual',
    params=parameters,
    credentials = Credentials(
    url=wxa_url,
    api_key=wxa_api_key,
),
    project_id=wxa_project_id
)

allam_llm_1.get_details()

### Elasticsearch method

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings

emb_func = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
<ipython-input-1-a996f72d8854>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  emb_func = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
# Defining model
model_id = ModelTypes.FLAN_UL2

In [ ]:
# Defining the model parameters
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 50
}

In [ ]:
# Initialize the Model class

model = Model(
    model_id="sdaia/allam-1-13b-instruct",
    credentials=Credentials(
        api_key = "9L8TsBVh597KH2qocdFOgBOFhwL93Rt-iayij5EYFyoM",
        url = "https://eu-de.ml.cloud.ibm.com"),
    project_id="64d518b6-f02e-4572-be32-cb1ba20cfd7a"
    )

### Basic information how to connect to Elasticsearch


In [ ]:
try:
    esuser = os.environ["ESUSER"]
except KeyError:
    esuser = input("Please enter your Elasticsearch user name (hit enter): ")
try:
    espassword = os.environ["ESPASSWORD"]
except KeyError:
    espassword = getpass.getpass("Please enter your Elasticsearch password (hit enter): ")
try:
    eshost = os.environ["ESHOST"]
except KeyError:
    eshost = input("Please enter your Elasticsearch hostname (hit enter): ")
try:
    esport = os.environ["ESPORT"]
except KeyError:
    esport = input("Please enter your Elasticsearch port number (hit enter): ")

Please enter your Elasticsearch user name (hit enter): 


NameError: name 'getpass' is not defined

In [ ]:
# Load the .env file
_ = load_dotenv(find_dotenv())

# Access the environment variables
api_key = os.getenv("IBM_WATSONX_API_KEY")

print(f"API Key: {api_key}")